In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate

In [1]:
# ---- 1. Your inputs & prompt ----

Section_Title = "Risk Section Title"
Section_Text = "Risk Section Text"

questions_market_dynamics = {
    "Market Dynamics - a": "Does the text mention that the company is exposed to risks associated with cyclical products?"
}

questions_intra_industry_competition = {
    "Intra-Industry Competition - a": "Does the text mention that market pricing for the company's products is irrational?"
}

all_question_dicts = [
    questions_market_dynamics,
    questions_intra_industry_competition
]

PROMPT = """{question} 
Title: {Section_Title} 
Text: {Section_Text}

Provide your answer in the following JSON format:
{{
  "Answer": "Yes" or "No", 
  "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}}
"""

# ---- 2. Load your LLM locally ----
# Requires `pip install langchain_community`
from langchain_community.llms import LlamaCpp

model_path = "../gguf_folder/Llama-3.2-3B-Instruct-Q8_0.gguf"
llm_hf = LlamaCpp(
    model_path=model_path,
    n_ctx=4096,
    n_gpu_layers=35,
    max_tokens=256
)

# ---- 3. Build a Runnable to format the prompt ----
# This Runnable takes a dict with keys: "question", "Section_Title", "Section_Text"
# and returns the fully formatted string prompt.

prompt_runnable = RunnableLambda(
    lambda x: PROMPT.format(
        question=x["question"],
        Section_Title=x["Section_Title"],
        Section_Text=x["Section_Text"]
    )
)

# ---- 4. Pipe prompt construction into your LLM Runnable ----
# The "pipe" operator (|) creates a RunnableSequence that:
#   (a) calls prompt_runnable.invoke(...)
#   (b) pipes its output into llm_hf.invoke(...)

chain = prompt_runnable | llm_hf

# ---- 5. Invoke chain for each question ----
if __name__ == "__main__":
    for question_dict in all_question_dicts:
        for column_name, question in question_dict.items():
            # Construct the input dict
            input_dict = {
                "Section_Title": Section_Title,
                "Section_Text": Section_Text,
                "question": question,
            }

            # Invoke the chain
            output = chain.invoke(input_dict)
            
            # Print or store results as desired
            print(f"Column Name: {column_name}")
            print("LLM Output:", output)
            print("-" * 50)

ImportError: Could not import llama-cpp-python library. Please install the llama-cpp-python library to use this embedding model: pip install llama-cpp-python